<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

from dotenv import load_dotenv
import os

load_dotenv()  # Загружаем переменные из .env

DBNAME = os.getenv("DBNAME")
USER = os.getenv("USER")
PASSWORD = os.getenv("PASSWORD")
HOST = os.getenv("HOST")
PORT = os.getenv("PORT")


In [3]:
# подключение через sqlalchemy
engine = create_engine(f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}', client_encoding='utf8')

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''select count(*) from vacancies'''

count_vacancies = pd.read_sql_query(query_3_1, engine)
count_vacancies

,count
0,49197


**Результат запроса**

Общее количество вакансий в базе **49197**

---

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [5]:
# текст запроса
query_3_2 = f'''select count(*) from employers'''
count_employers = pd.read_sql_query(query_3_2, engine)
count_employers

,count
0,23501


**Результат запроса**

Общее количество работодатлей **23501**

---

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [9]:
# текст запроса

query_3_3 = f'''select count(*) from areas'''
count_areas = pd.read_sql_query(query_3_3, engine)
count_areas

,count
0,1362


**Результат запроса**

Общее количество представленных регионов **1362**

---

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса

query_3_4 = f'''select count(*) from industries'''
count_industries = pd.read_sql_query(query_3_4, engine)
count_industries

,count
0,294


**Результат запроса**

Общее количество сфер деятельности **294**

---

## Выводы по предварительному анализу данных

В базе данных представлено 49197 вакансий, которые размещены 23501 работодателем. Вакансии охватывают 1362 региона и 294 сферы деятельности. 

Это говорит о том, что работодатели в среднем публикуют по 2 вакансии, что может означать, что у большинства компаний мало открытых позиций. Но это пока предварительно, надо посмотреть распределение числа вакансий по работодателям.

Количество регионов довольно велико, значит регионы охватывают как крупные города, так и небольшие населенные пункты. 

Также видно по значительному количеству сфер деятельности, что рынок вакансий достаточно диверсифицирован.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [7]:
# текст запроса

query_4_1 = f'''
-- Выбираем название региона и количество вакансий в нем
SELECT  
    a.name AS area,     -- Название региона
    COUNT(*) AS cnt     -- Количество вакансий
FROM areas a
JOIN vacancies v
    ON a.id = v.area_id  -- Связываем таблицу вакансий с регионами по ID
GROUP BY 1               -- Группируем по названию региона
ORDER BY 2 DESC          -- Сортируем по количеству вакансий (по убыванию)
LIMIT 15                 -- Оставляем только 15 регионов с наибольшим числом вакансий
'''

# Выполняем запрос и загружаем данные в DataFrame
count_first_five = pd.read_sql_query(query_4_1, engine)
count_first_five

# результат запроса

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [8]:
# текст запроса

query_4_2 = f'''
-- Считаем количество вакансий, у которых указана хотя бы одна граница зарплатного диапазона
SELECT  
    COUNT(*) AS vacancies_with_salary  -- Количество вакансий с указанной зарплатой
FROM vacancies
WHERE salary_from IS NOT NULL 
   OR salary_to IS NOT NULL  -- Учитываем вакансии, где указана хотя бы одна граница зарплаты
'''

# Выполняем запрос и загружаем результат в DataFrame
vacancies_with_salary = pd.read_sql_query(query_4_2, engine)
vacancies_with_salary

# результат запроса

,vacancies_with_salary
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [9]:
# текст запроса

query_4_3 = f'''
-- Определяем средние значения для нижней и верхней границы зарплатного диапазона
-- Округляем до целого числа
SELECT  
    ROUND(AVG(salary_from)) AS avg_salary_from,  -- Средняя нижняя граница зарплаты
    ROUND(AVG(salary_to)) AS avg_salary_to      -- Средняя верхняя граница зарплаты
FROM vacancies
'''

# Выполняем запрос и загружаем результат в DataFrame
avg_salaries = pd.read_sql_query(query_4_3, engine)
avg_salaries

# результат запроса

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# текст запроса

query_4_4 = f'''
-- Подсчитываем количество вакансий для каждого сочетания 
-- типа рабочего графика (schedule) и типа трудоустройства (employment)
SELECT  
    schedule,      -- Тип рабочего графика (полный день, гибкий, сменный и т. д.)
    employment,    -- Тип занятости (полная, частичная, стажировка и т. д.)
    COUNT(*) AS vacancy_count  -- Количество вакансий для данной комбинации
FROM vacancies 
GROUP BY schedule, employment  -- Группируем по обоим параметрам
ORDER BY vacancy_count DESC     -- Сортируем по количеству вакансий в порядке убывания
'''

# Выполняем запрос и загружаем результат в DataFrame
schedule_employment_stats = pd.read_sql_query(query_4_4, engine)
schedule_employment_stats


# результат запроса

,schedule,employment,vacancy_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [ ]:
# текст запроса

query_4_5 = f'''
-- Подсчитываем количество вакансий для каждого варианта требуемого опыта работы
SELECT  
    experience,      -- Требуемый опыт работы (например, "Нет опыта", "От 1 года до 3 лет" и т. д.)
    COUNT(*) AS vacancy_count  -- Количество вакансий с данным уровнем опыта
FROM vacancies 
GROUP BY experience  -- Группируем по полю "experience"
ORDER BY vacancy_count ASC  -- Сортируем по количеству вакансий в порядке возрастания
'''

# Выполняем запрос и загружаем результат в DataFrame
experience_stats = pd.read_sql_query(query_4_5, engine)
experience_stats


# результат запроса

,experience,vacancy_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

## выводы по детальному анализу вакансий

### **1. Количество вакансий в регионах**  
- Наибольшее количество вакансий представлено в **Москве (5333)**, затем идут **Санкт-Петербург (2851)** и **Минск (2112)**.  
- Высокая активность наблюдается, в основном, в столицах  и крупных городах.  
- Среди зарубежных городов с наибольшим количеством выставленных вакансий:  **Минск (2112), Алматы(1892), Ташкент (851), Нур-Султан (840)**, что говорит о том, что международный рынок труда представляют, главным образом, страны ближнего зарубежья.

---

### **2. Заполненность полей зарплат**  
- У **24 073 из 49 197 вакансий (около 49%)** есть хотя бы одно из двух значений зарплаты (`salary_from` или `salary_to`).  
- Это означает, что почти половина работодателей **указывает уровень дохода**, но значительная часть (примерно 51%) предпочитает не раскрывать зарплатные ожидания.  
- Это может указывать на гибкость в переговорах о зарплате или нежелание заранее показывать возможный уровень дохода соискателям.

---

### **3. Средний уровень зарплатной вилки**  
- Среднее значение **`salary_from`**: **71065 ₽**.  
- Среднее значение **`salary_to`**: **110537 ₽**.  
- Средний разброс **39472 ₽**, что указывает на значительный диапазон зарплатных предложений.  
- Верхняя граница (`salary_to`) почти в **1.5 раза выше**, что может свидетельствовать о взаимозависимости потенциальной зарплаты и квалификации кандидата.

---

### **4. Сочетание графика работы и занятости**  
- **Полный день + Полная занятость (35367 вакансий, 72%)** — самый популярный формат работы.  
- **Удаленная работа + Полная занятость (7802 вакансии, около 16%)** — удаленная работа также довольно активно представлена и востребована.  
- **Гибкий график и сменные режимы (1593 и 940 вакансий)**  встречаются реже.  
- **Вахтовый метод (367 вакансий) и стажировки (569+116+64 = 749 вакансий)** — очень ограниченные предложения, но они есть.  
- Количество вакансий на проектную работу **самое низкое** (от 1 до 141).

Из всего этого можно сделать вывод о значительном преобладании спроса на **постоянную занятость** на рынке.

---

### **5. Требуемый опыт работы**  
- Вакансии распределены следующим образом:  
  - **Более 6 лет — 1337 (самая редкая категория, 2.7%)**  
  - **Нет опыта — 7197 (около 14.6%)**  
  - **От 3 до 6 лет — 14 511 (примерно 29.5%)**  
  - **От 1 года до 3 лет — 26 152 (около 53.2%)**  

 **Это говорит о том, что:**  
- Больше половины вакансий (53%) ориентированы на кандидатов с опытом работы **1-3 года**.  
- 29% вакансий требуют **от 3 до 6 лет опыта работы**, что показывает востребованность **средне-квалифицированных специалистов**.  
- Всего **2.7% вакансий требуют более 6 лет опыта**, что говорит о небольшом количестве позиций для высококвалифицированных специалистов.  
- Доля вакансий **без опыта (14.6%)** говорит о наличии позиций для начинающих, но они занимают **небольшую часть рынка**.  

---

### **Ключевые  выводы по вакансиям**  
* **Рабочие места сосредоточены в крупных городах, особенно в Москве и Санкт-Петербурге.**  
* **Зарплата указывается примерно в 49% вакансий, в среднем от 71 000 до 110 000 ₽.**  
* **Полная занятость и стандартный рабочий день — наиболее распространенный формат работы (72% вакансий).**  
* **Удаленная работа занимает 16% рынка, гибкие и сменные графики встречаются реже.**  
* **Более половины работодателей ищут кандидатов с опытом 1-3 года, вакансии для специалистов с опытом более 6 лет редки (2.7%).**  

---

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [5]:
# текст запроса

query = f'''
-- Определяем работодателей с наибольшим количеством вакансий
SELECT  
    e.name AS employer_name,  -- Название работодателя
    COUNT(*) AS vacancy_count  -- Количество вакансий данного работодателя
FROM employers e
JOIN vacancies v 
ON e.id = v.employer_id
GROUP BY e.name  -- Группируем по названию работодателя
ORDER BY vacancy_count DESC  -- Сортируем по убыванию количества вакансий
LIMIT 5  -- Оставляем только топ-5 работодателей
'''

# Выполняем запрос и загружаем результат в DataFrame
df = pd.read_sql_query(query, engine)
print(df, "\n")

# Выводим первого и пятого работодателя из результата
top_1 = df.iloc[0]  # Первый работодатель
top_5 = df.iloc[4]  # Пятый работодатель


print("На первом месте:\n", top_1,"\n") 

print("На пятом месте:\n", top_5) 



   employer_name  vacancy_count
0         Яндекс           1933
1     Ростелеком            491
2       Тинькофф            444
3           СБЕР            428
4  Газпром нефть            331 

На первом месте:
 employer_name    Яндекс
vacancy_count      1933
Name: 0, dtype: object 

На пятом месте:
 employer_name    Газпром нефть
vacancy_count              331
Name: 4, dtype: object


**Результат запроса**

На первом месте по количеству размещенных вакансий находится компания "Яндекс".
На пятом - компания "Газпром нефть"

---

2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. 


**Коммнентарий к заданию 2.** 

В БД есть две категории работодателей:
1. Работодатели, **зарегистрированные** в различных регионах (таблица `employers`), и
2. Работодатели, **разместившие вакансии** в различных регионах (таблица `vacancies`)

В связи с этим условие Задания 2 звучит неопределенно. 
- В первой части задания для каждого региона надо посчитать количество вакансий и, видимо, количество работодателей, **разместивших вакансии** в этом регионе. 
- Во второй части надо определить регион, в котором не размещено ни одной вакансии и **зарегистрировано** максимальное количество работодателей. 
 
Поэтому для  для выполнения задания нужны два отдельных запроса. Так как, если делать один запрос, то нет никакого смысла **считать количество вакансий** в регионе (первая часть), в котором их не должно быть изначально (вторая часть). Это количество для каждого региона будет равно нулю. 

In [9]:
# текст запроса

query_2_1 = f''' -- Запрос 1: Подсчет количества работодателей и вакансий в каждом регионе
    SELECT  
        a.name AS area_name,               -- Название региона        
        COUNT(DISTINCT v.employer_id) AS count_employers,  -- Количество уникальных работодателей, разместивших вакансии в регионе
        COUNT(v.id) AS count_vacancies     -- Количество вакансий
    FROM areas a
    JOIN vacancies v                  
    ON a.id = v.area_id
    GROUP BY a.name
    ORDER BY  count_vacancies DESC
'''

query_2_2 = f'''-- Запрос 2: Поиск регионов без вакансий и подсчет числа работодателей в них
    SELECT  
        a.name AS area_name,  -- Название региона
        count(distinct e.id) AS count_employers  -- Количество работодателей, зарегистрированных в регионе
    FROM areas a                               
    LEFT JOIN vacancies v ON a.id = v.area_id -- Используем LEFT JOIN, чтобы включить регионы без вакансий
    LEFT JOIN employers e ON a.id = e.area -- Соединяем с таблицей работодателей
    WHERE v.id IS NULL  -- Оставляем только регионы, в которых нет вакансий
    GROUP BY area_name  
    ORDER BY count_employers DESC;  -- Сортируем по убыванию количества работодателей
'''



# Выполняем запросы и загружаем результат в DataFrame
df = pd.read_sql_query(query_2_1, engine)
print(f'Количество работодателей и вакансий по регионам')
display(df)

df = pd.read_sql_query(query_2_2, engine)
print(f'Количество работодателей в регионах без вакансий')
display(df)

print(f'Регион без вакансий, в котором зарегистрировано наибольшее количество работодателей: {df.iloc[0,0]}')
print(f'Количество работодателей {df.iloc[0,1]}')



Количество работодателей и вакансий по регионам


,area_name,count_employers,count_vacancies
0,Москва,2320,5333
1,Санкт-Петербург,1451,2851
2,Минск,851,2112
3,Новосибирск,802,2006
4,Алматы,798,1892
...,...,...,...
764,Шахтерск (Сахалинская область),1,1
765,Пестово,1,1
766,Пересвет,1,1
767,Первомайское (Ленинградская область),1,1


Количество работодателей в регионах без вакансий


,area_name,count_employers
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Беларусь,18
...,...,...
588,Кувандык,0
589,Куйбышев,0
590,Куйбышево (Ростовская область),0
591,Култаево,0


Регион без вакансий, в котором зарегистрировано наибольшее количество работодателей: Россия
Количество работодателей 410


**Результат запроса**

Количество работодателей и вакансий в каждом регионе представлено в результирующей таблице, полученной по запросу query_2_1.

Название региона, в котором зарегистрировано наибольшее количество работодателей и не размещено ни одной вакансии, - `Россия`.

---

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [13]:
# текст запроса

# SQL-запрос для подсчёта количества регионов, в которых работодатель публикует вакансии
query = """
    SELECT e.name,  -- Название работодателя
           COUNT(DISTINCT v.area_id) AS region_count  -- Количество уникальных регионов
    FROM employers e
    JOIN vacancies v ON e.id = v.employer_id  -- Связываем таблицы по ID работодателя
    GROUP BY e.name  -- Группируем по названию работодателя
    ORDER BY region_count DESC  -- Сортируем по убыванию количества регионов
"""

# Выполняем запрос и загружаем результат в DataFrame
df = pd.read_sql(query, engine)

# Вывод результата
df


,name,region_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [14]:
# текст запроса

# SQL-запрос для подсчёта количества работодателей, у которых не указана сфера деятельности
query = """
    SELECT COUNT(e.id) AS employer_count  -- Подсчитываем количество таких работодателей
    FROM employers e
    LEFT JOIN employers_industries e_i
    ON e.id = e_i.employer_id
    WHERE e_i.industry_id IS NULL  -- Оставляем только тех, у кого нет сферы деятельности
"""

# Выполняем запрос и загружаем результат в DataFrame
df = pd.read_sql(query, engine)

# Вывод результата
df


,employer_count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [4]:
# текст запроса

# SQL-запрос для получения названия компании, находящейся на третьем месте
# в алфавитном списке компаний, у которых указано четыре сферы деятельности
query = """
    SELECT e.name  -- Название компании
    FROM employers e
    JOIN employers_industries e_i ON e.id = e_i.employer_id  -- Связываем таблицы
    GROUP BY e.name  -- Группируем по названию компании
    HAVING COUNT(e_i.industry_id) = 4  -- Оставляем только компании с четырьмя сферами
    ORDER BY e.name  -- Сортируем по алфавиту
    LIMIT 1 OFFSET 2  -- Пропускаем две записи и берём третью
"""

# Выполняем запрос и загружаем результат в DataFrame
df = pd.read_sql(query, engine)

# Вывод результата
df

,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [5]:
# текст запроса

# SQL-запрос для определения количества работодателей, 
# у которых указана сфера "Разработка программного обеспечения"
query = """
    SELECT COUNT(*) AS employer_count  -- Подсчитываем количество таких работодателей
    FROM industries i
    JOIN employers_industries ei ON i.id = ei.industry_id  -- Связываем таблицы по ID сферы
    WHERE i.name = 'Разработка программного обеспечения'  -- Фильтруем по названию сферы
"""

# Выполняем запрос и загружаем результат в DataFrame
df = pd.read_sql(query, engine)

# Вывод результата
df


,employer_count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [10]:
# код для получения списка городов-милионников

import requests
from bs4 import BeautifulSoup
# URL страницы
url = "https://ru.wikipedia.org/wiki/Города-миллионеры_России"

# Добавляем заголовки, чтобы имитировать реальный браузер
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# Загружаем страницу
response = requests.get(url, headers=headers)
page = BeautifulSoup(response.text, "html.parser")


# Находим таблицу с названиями городов по классам 'standard sortable'
table = page.find("table", class_="standard sortable")

# Извлекаем все строки таблицы, пропуская шапку
rows = table.find_all("tr")[1:]  

# Извлекаем названия городов 
# (вторая ячейка в каждой строке содержит тег ссылки с названием города)
city_names = [
    row.find_all("td")[1].find("a").text
    for row in rows    
]
# Выводим список городов
print(city_names)



['Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград']


In [15]:
# текст запроса

# Преобразуем список городов-миллионников в строку с кавычками для SQL запроса
city_names_str = ", ".join(f"'{city}'" for city in city_names)

# SQL-запрос для получения списка регионов-миллионников, в которых представлены вакансии Яндекса
query = f"""
WITH yandex_vacancies AS (
    SELECT a.name AS region_name, COUNT(*) AS vacancy_count
    FROM areas a
    JOIN vacancies v ON a.id = v.area_id
    JOIN employers e ON v.employer_id = e.id
    WHERE e.name = 'Яндекс' 
      AND a.name IN ({city_names_str})  -- Фильтр по городам-миллионникам
    GROUP BY a.name  -- Группируем по названию региона
)
-- Получаем список регионов и количество вакансий Яндекс в них, 
-- сортируем по возрастанию
(SELECT region_name, vacancy_count FROM yandex_vacancies
ORDER BY vacancy_count DESC)
UNION ALL
-- Добавляем строку Total с общим количеством вакансий
SELECT 'Total', SUM(vacancy_count) FROM yandex_vacancies;
"""

# Выполняем запрос и загружаем результат в DataFrame
df = pd.read_sql(query, engine)

# Вывод результата
df


,region_name,vacancy_count
0,Москва,54.0
1,Санкт-Петербург,42.0
2,Екатеринбург,39.0
3,Нижний Новгород,36.0
4,Новосибирск,35.0
5,Воронеж,32.0
6,Краснодар,30.0
7,Самара,26.0
8,Уфа,26.0
9,Казань,25.0


---

## **Вывод по анализу работодателей и вакансий**  

### **1. Количество вакансий у работодателей**  
Лидером по количеству вакансий является **Яндекс** (1933 вакансии), за ним следуют **Ростелеком** (491), **Тинькофф** (444), **СБЕР** (428) и **Газпром нефть** (331). Это свидетельствует о высокой потребности в кадрах в крупнейших российских компаниях, особенно в IT-секторе и банковской сфере.  

### **2. Распределение работодателей и вакансий по регионам**  

Наибольшее количество вакансий и работодателей сосредоточено в крупных городах:  
- **Москва** (2320 работодателей, 5333 вакансии)  
- **Санкт-Петербург** (1451 работодателей, 2851 вакансии)  
- **Минск** (851 работодателей, 2112 вакансии)  
- **Новосибирск** (802 работодателей, 2006 вакансии)  

Однако существуют регионы, где зарегистрировано много работодателей, но нет ни одной вакансии. Например, в **России** без привязки к конкретному региону зарегистрировано 410 работодателей, в **Казахстане** – 207, а в **Московской области** – 75.  

### **3. География деятельности работодателей**  
Наибольший охват по количеству регионов, в которых размещены вакансии, имеет **Яндекс** (181 регион), далее идут **Ростелеком** (152) и **Спецремонт** (116). Это говорит о тенденции к территориальному расширению деятельности этих компаний и растущую, в связи с этим, потребность в специалистах по всей стране.  

### **4. Работодатели без указанной сферы деятельности**  
У **8419 работодателей** отсутствует информация о сфере деятельности. Это может быть связано с недостаточностью предоставленных данных или спецификой бизнеса некоторых компаний.  

### **5. Работодатели с четырьмя сферами деятельности**  
На третьем месте в алфавитном списке среди компаний, у которых указано **четыре сферы деятельности**, находится **2ГИС**.  

### **6. Количество работодателей в сфере "Разработка программного обеспечения"**  
Сфера **"Разработка ПО"** представлена **3553 работодателями**, что подтверждает высокий спрос на IT-специалистов.  

### **7. Вакансии Яндекса в городах-миллионниках**  
Компания **Яндекс** активно размещает вакансии в крупнейших городах России. Больше всего вакансий в:  
- **Москве** (54 вакансии),  
- **Санкт-Петербурге** (42),  
- **Екатеринбурге** (39).  

Общее количество вакансий Яндекса в городах-миллионниках составило **485**. 

---

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [16]:
# текст запроса

# SQL-запрос для подсчета вакансий, имеющих отношение к данным
query = f"""
WITH names_grouped AS (
    -- Выбираем название вакансии и считаем количество повторений
    SELECT name, COUNT(*) AS count
    FROM vacancies
    -- Учитываем вакансии, в названии которых встречаются 'data' или 'данн' (в любом регистре)
    WHERE name ILIKE '%%data%%' OR name ILIKE '%%данн%%' -- экранируем знак процента
    GROUP BY name
)
-- Суммируем количество таких вакансий
SELECT SUM(count)
FROM names_grouped
"""

# Выполняем SQL-запрос и загружаем результат в DataFrame
df = pd.read_sql(query, engine)

# Выводим результат
df


,sum
0,1771.0


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [17]:
# текст запроса

# SQL-запрос для подсчета вакансий, подходящих для начинающего дата-сайентиста
query = """
WITH data_science_vacancies AS (
    -- Выбираем вакансии, относящиеся к Data Science
    SELECT name, experience, employment
    FROM vacancies
    WHERE 
        -- Фильтр по ключевым словам для Data Science
        name ILIKE ANY(%(keywords)s) 
        -- Фильтр по 'ML', исключая вакансии, связанные с HTML
        OR (name ILIKE %(ML)s AND name NOT ILIKE %(HTML)s)
)
-- Отбираем только вакансии для Junior-специалистов
SELECT COUNT(*)
FROM data_science_vacancies
WHERE 
    name ILIKE %(junior)s  -- В названии есть слово "junior"
    OR experience = 'Нет опыта'  -- Требуемый опыт — "Нет опыта"
    OR employment = 'Стажировка'  -- Тип трудоустройства — "Стажировка"
"""

# Список ключевых слов для фильтрации Data Science вакансий
keywords = [
    '%data scientist%', 
    '%data science%', 
    '%исследователь данных%', 
    '%machine learning%', 
    '%машинн%обучен%'
]

# Параметры для подстановки в SQL-запрос
params = {
    "keywords": keywords, 
    "ML": "%ML%", 
    "HTML": "%HTML%", 
    "junior": "%junior%"
}

# Выполняем SQL-запрос и загружаем результат в DataFrame
df = pd.read_sql(query, engine, params=params)

# Выводим результат
df


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [18]:
# текст запроса

# SQL-запрос для подсчета вакансий в Data Science, где среди ключевых навыков указан SQL или PostgreSQL
query = """
WITH data_science_vacancies AS (
    -- Выбираем вакансии, относящиеся к Data Science
    SELECT key_skills
    FROM vacancies
    WHERE 
        -- Фильтр по ключевым словам для Data Science
        name ILIKE ANY(%(keywords)s) 
        -- Фильтр по 'ML', исключая вакансии, связанные с HTML
        OR (name ILIKE %(ML)s AND name NOT ILIKE %(HTML)s)
)
-- Отбираем только те вакансии, где среди ключевых навыков есть SQL или PostgreSQL
SELECT COUNT(*)
FROM data_science_vacancies
WHERE 
    key_skills ILIKE %(skill1)s  -- Навык SQL
    OR key_skills ILIKE %(skill2)s  -- Навык PostgreSQL
"""

# Список ключевых слов для фильтрации Data Science вакансий
keywords = [
    '%data scientist%', 
    '%data science%', 
    '%исследователь данных%', 
    '%machine learning%', 
    '%машинн%обучен%'
]

# Параметры для подстановки в SQL-запрос
params = {
    "keywords": keywords, 
    "ML": "%ML%", 
    "HTML": "%HTML%", 
    "skill1": "%SQL%", 
    "skill2": "%postgres%"
}

# Выполняем SQL-запрос и загружаем результат в DataFrame
df = pd.read_sql(query, engine, params=params)

# Выводим результат
df


,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [19]:
# текст запроса

# SQL-запрос для подсчета вакансий в Data Science, где среди ключевых навыков указан Python
query = """
WITH data_science_vacancies AS (
    -- Выбираем вакансии, относящиеся к Data Science
    SELECT key_skills
    FROM vacancies
    WHERE 
        -- Фильтр по ключевым словам для Data Science
        name ILIKE ANY(%(keywords)s) 
        -- Фильтр по 'ML', исключая вакансии, связанные с HTML
        OR (name ILIKE %(ML)s AND name NOT ILIKE %(HTML)s)
)
-- Отбираем только те вакансии, где среди ключевых навыков есть Python
SELECT COUNT(*)
FROM data_science_vacancies
WHERE key_skills ILIKE %(skill)s
"""

# Список ключевых слов для фильтрации Data Science вакансий
keywords = [
    '%data scientist%', 
    '%data science%', 
    '%исследователь данных%', 
    '%machine learning%', 
    '%машинн%обучен%'
]

# Параметры для подстановки в SQL-запрос
params = {
    "keywords": keywords, 
    "ML": "%ML%", 
    "HTML": "%HTML%", 
    "skill": "%Python%"
}

# Выполняем SQL-запрос и загружаем результат в DataFrame
df = pd.read_sql(query, engine, params=params)

# Выводим результат
df


,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [20]:
# текст запроса

# SQL-запрос для расчета среднего количества ключевых навыков в вакансиях Data Science
query = """
WITH data_science_vacancies AS (
    -- Выбираем вакансии, относящиеся к Data Science, где указаны ключевые навыки
    SELECT key_skills
    FROM vacancies v
    WHERE 
        -- Фильтр по ключевым словам для Data Science
        v.name ILIKE ANY(%(keywords)s) 
        -- Фильтр по 'ML', исключая вакансии, связанные с HTML
        OR (v.name LIKE %(ML)s AND v.name NOT ILIKE %(HTML)s)
        -- Исключаем вакансии, где нет ключевых навыков, но можно этот фильтр и не ставить,
        -- функция AVG() далее проигнорирует все NULL автоматом 
        AND v.key_skills IS NOT NULL
)
-- Рассчитываем среднее количество ключевых навыков
SELECT ROUND(
    AVG(
        LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1
    ), 2) AS avg_skills
FROM data_science_vacancies
"""

# Список ключевых слов для фильтрации Data Science вакансий
keywords = [
    '%data scientist%', 
    '%data science%', 
    '%исследователь данных%', 
    '%machine learning%', 
    '%машинн%обучен%'
]

# Параметры для подстановки в SQL-запрос
params = {
    "keywords": keywords, 
    "ML": "%ML%", 
    "HTML": "%HTML%"
}

# Выполняем SQL-запрос и загружаем результат в DataFrame
df = pd.read_sql(query, engine, params=params)

# Выводим результат
df


,avg_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [35]:
# текст запроса

# SQL-запрос для расчета средней зарплаты для DS в разрезе опыта работы
query = """
SELECT experience, 
       -- Рассчитываем среднюю зарплату с округлением до целого числа
       ROUND(AVG(
           -- Если указаны обе границы зарплаты, берем их среднее
           COALESCE((salary_from + salary_to) / 2, 
           -- Если указана только одна граница, берем ее
           salary_from, salary_to)
       )) AS average_salary
FROM vacancies
WHERE 
    -- Фильтр по ключевым словам для Data Science
    (name ILIKE ANY(%(keywords)s) 
    OR (name LIKE %(ML)s AND name NOT ILIKE %(HTML)s))
    -- Берем только те вакансии, где указана хотя бы одна из зарплат
    AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
GROUP BY experience
ORDER BY experience
"""

# Список ключевых слов для фильтрации Data Science вакансий
keywords = [
    '%data scientist%', 
    '%data science%', 
    '%исследователь данных%', 
    '%machine learning%', 
    '%машинн%обучен%'
]

# Параметры для подстановки в SQL-запрос
params = {
    "keywords": keywords, 
    "ML": "%ML%", 
    "HTML": "%HTML%"
}

# Выполняем SQL-запрос и загружаем результат в DataFrame
df = pd.read_sql(query, engine, params=params)

# Выводим результат запроса
display(df)

# Фильтруем только вакансии с опытом "От 3 до 6 лет"
df = df[df['experience'] == 'От 3 до 6 лет']

# Выводим результат
df



,experience,average_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


,experience,average_salary
2,От 3 до 6 лет,243115.0


***

## Выводы по предметному анализу

---

### **Задание 6.1: Сколько вакансий имеет отношение к данным?**  
**Результат**: 1771 вакансия.  
**Вывод**: Вакансий, связанных с данными, довольно много, что подтверждает высокий спрос на специалистов в этой области.  

---

### **Задание 6.2: Сколько есть подходящих вакансий для начинающего дата-сайентиста?**  
**Результат**: 51 вакансия.  
**Вывод**: Количество вакансий для Junior-специалистов относительно небольшое (2.88% от всех вакансий, связанных с данными). Это говорит о том, что работодатели чаще ищут более опытных кандидатов, а начинающим специалистам может быть сложнее найти работу.  

---

### **Задание 6.3: Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?**  
**Результат**: 229 вакансий.  
**Вывод**: SQL и Postgres востребованы примерно в 12.9% вакансий для DS. Это подтверждает, что умение работать с базами данных — важный навык для дата-сайентиста.  

---

### **Задание 6.4: Насколько популярен Python в требованиях работодателей к DS?**  
**Результат**: 357 вакансий.  
**Вывод**: Python встречается в требованиях примерно в 20.2% вакансий. Это делает его одним из самых популярных языков программирования среди требований к дата-сайентистам.  

---

### **Задание 6.5: Сколько ключевых навыков в среднем указывают в вакансиях для DS?**  
**Результат**: 6.41 навыка.  
**Вывод**: В среднем работодатели ожидают, что кандидат по DS-вакансиям обладает более чем шестью разными навыками. Это подчеркивает важность владения широким спектром инструментов и технологий.  

---

### **Задание 6.6: Какая средняя зарплата для DS в зависимости от опыта?**  
**Результаты**:  
- **Нет опыта** - 74 643  
- **От 1 до 3 лет** - 139 675  
- **От 3 до 6 лет** - 243 115  

**Вывод**:  
Зарплата растет с увеличением опыта, причем скачок между уровнями "От 1 до 3 лет" и "От 3 до 6 лет" довольно значительный. Это говорит о том, что приобретение опыта и развитие навыков в области DS значительно повышает доход.  

---
**Общий вывод**

Анализ показывает, что:  
1. **Сфера науки о данных очень востребована** – более 1700 вакансий содержат слова "data" или "данные".  
2. **Начинающим специалистам сложнее устроиться** – вакансий для Junior всего 51.  
3. **SQL (229 упоминаний) и Python (357 упоминаний) – ключевые навыки** в DS.  
4. **Специалисты должны обладать широким спектром навыков** (в среднем 6.41 навыка).  
5. **Зарплата значительно растет с опытом** – специалисты с 3-6 годами опыта зарабатывают в 3.2 раза больше, чем те, у кого опыта нет.  

Этот анализ подтверждает, что Data Science – перспективная, востребованная профессия с высокими требованиями к кандидатам, где важно не только знание языков программирования, но и работа с базами данных, машинное обучение и аналитические навыки.

# Общий вывод по проекту

##### подведем итог исследования, обобщите выводы
##### здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

## **Общий вывод по проекту "Анализ вакансий из HeadHunter"**

Анализ данных о вакансиях с HeadHunter позволил выявить ключевые тенденции на рынке труда, особенности занятости, требования к кандидатам и востребованные навыки.

### **1. Общие характеристики базы вакансий**
В базе представлено 49 197 вакансий от 23 501 работодателя, охватывающих 1 362 региона и 294 сферы деятельности. Это указывает на широкий географический охват рынка труда и его диверсификацию.

Среднее число вакансий на одного работодателя – около 2, что может свидетельствовать о небольшом количестве открытых позиций в большинстве компаний. Однако распределение числа вакансий требует детального анализа.

### **2. География и распределение вакансий**
- Наибольшее количество вакансий представлено в **Москве (5333 вакансии)**, **Санкт-Петербурге (2851)** и **Минске (2112)**.
- Высокая активность работодателей наблюдается в крупных городах и столицах.
- В странах ближнего зарубежья значительное число вакансий размещено в **Алматы, Ташкенте и Нур-Султане**.

### **3. Зарплатные ожидания**
- Лишь в **49% вакансий указана зарплата**.
- Средний уровень зарплатной вилки: **от 71 065 ₽ до 110 537 ₽**, с разбросом в **39 472 ₽**.
- Верхняя граница зарплаты почти в 1.5 раза выше нижней, что может отражать зависимость заработка от квалификации кандидата.

### **4. Форматы работы и занятость**
- **Полная занятость и полный рабочий день** преобладают (72% вакансий).
- **Удаленная работа** составляет 16% вакансий, подтверждая тенденцию к распространению гибких форматов работы.
- **Гибкий график, сменные режимы, стажировки и вахтовый метод** представлены в меньшей степени.

### **5. Требования к опыту работы**
- **53.2% вакансий требуют 1-3 года опыта**.
- **29.5% – от 3 до 6 лет**.
- Лишь **2.7% вакансий требуют более 6 лет опыта**, что говорит о небольшом количестве позиций для высококвалифицированных специалистов.
- 14.6% вакансий предназначены для кандидатов без опыта работы.

### **6. Анализ работодателей**
- Лидерами по числу вакансий являются **Яндекс (1933 вакансии), Ростелеком (491), Тинькофф (444), СБЕР (428), Газпром нефть (331)**.
- **Разработка ПО** – наиболее представленная сфера деятельности (3553 работодателя).
- Многие работодатели работают в нескольких сферах бизнеса, что подтверждает диверсификацию компаний.

### **7. Предметный анализ (Data Science и IT-направление)**
- Вакансий, связанных с данными, **1771**, что подтверждает высокую востребованность специалистов по анализу данных.
- Вакансий для **Junior Data Scientist** – **всего 51**, что говорит о сложностях входа в профессию.
- **SQL (229 вакансий) и Python (357 вакансий)** – наиболее востребованные технические навыки.
- Средний кандидат по DS-вакансии должен обладать **6.41 ключевым навыком**.
- Зарплаты растут с опытом: **от 74 643 ₽ (без опыта) до 243 115 ₽ (3-6 лет опыта)**.



### **Общий итог**

Анализ вакансий на HeadHunter показал, что рынок труда динамичен и активно развивается.

* Основной спрос сосредоточен в крупных городах, а наибольшее количество вакансий предлагают IT-компании.
* Удаленная работа набирает популярность, но пока не доминирует.
* Большинство работодателей предпочитает нанимать кандидатов с опытом работы от 1 года, а позиции для  Junior-специалистов составляют лишь небольшую часть рынка.
* Data Science – перспективное направление, но требует широкого набора навыков, а зарплаты сильно зависят от уровня опыта.
  

Анализ позволил выявить основные тренды и востребованные навыки, что может помочь как работодателям в поиске специалистов, так и соискателям в выборе карьерного пути.

## **Дополнительные исследования данных** 

### 1. Какие отрасли больше всего нуждаются в специалистах?

In [45]:
# SQL-запрос для определения отраслей с наибольшим числом вакансий
query = '''
    SELECT i.name, COUNT(*)  -- Выбираем название отрасли и количество вакансий
    FROM employers e
    JOIN employers_industries ei ON e.id = ei.employer_id  -- Связываем работодателей с их отраслями
    JOIN industries i ON ei.industry_id = i.id  -- Связываем с таблицей отраслей
    JOIN vacancies v ON v.employer_id = e.id  -- Связываем с вакансиями, чтобы учитывать только активных работодателей
    GROUP BY i.name  -- Группируем по названию отрасли
    ORDER BY COUNT(*) DESC  -- Сортируем по количеству вакансий по убыванию
    LIMIT 20  -- Ограничиваем выборку 15 самыми востребованными отраслями
'''

# Выполняем запрос и загружаем результат в DataFrame
df = pd.read_sql_query(query, engine)

# Выводим данные в удобочитаемом формате
for index, row in df.iterrows():
    print(f"{index + 1}. {row['name']} ({row['count']})")



1. Разработка программного обеспечения (12499)
2. Системная интеграция,  автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг (11034)
3. Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее) (6413)
4. Банк (2742)
5. Мобильная связь (1585)
6. Кадровые агентства (1143)
7. Консалтинговые услуги (1137)
8. Интернет-провайдер (1095)
9. Розничная сеть (продуктовая) (1034)
10. Фиксированная связь (1028)
11. Оптоволоконная связь (1012)
12. Маркетинговые, рекламные, BTL, дизайнерские, Event-, PR-агентства, организация выставок (1001)
13. Интернет-магазин (788)
14. Электронно-вычислительная, оптическая, контрольно-измерительная техника, радиоэлектроника, автоматика (продвижение, оптовая торговля) (622)
15. Электронно-вычислительная, оптическая, контрольно-измерительная техника, радиоэлектроника, автоматика (производство) (584)
16. Нефтепереработка, нефтехимия (производство) (573)
17. 

### **Выводы по анализу отраслей с наибольшим числом вакансий**  

**1. Самая востребованная сфера — IT и технологии**  
   - **Разработка программного обеспечения (12 499 вакансий)**,  
   - **Системная интеграция и автоматизация (11 034 вакансии)**,  
   - **Интернет-компании (6 413 вакансий)**.  
   Это подтверждает, что **спрос на IT-специалистов огромен**, и рынок продолжает активно расти. Разработка ПО, автоматизация бизнес-процессов, интеграция IT-решений и интернет-сервисы требуют большого количества сотрудников.  

**2. Финансовый сектор (банки) также остается одним из крупнейших работодателей**  
   - **Банки (2 742 вакансии)** занимают 4-е место.  
    
   Это говорит о **высоком спросе на специалистов в финансовой сфере**, особенно в области цифровых решений и банковских технологий.  

**3. Сфера телекоммуникаций и связи**  
   - **Мобильная связь (1 585 вакансий)**,  
   - **Интернет-провайдеры (1 095 вакансий)**,  
   - **Фиксированная связь (1 028 вакансий)**,  
   - **Оптоволоконная связь (1 012 вакансий)**.  
   
   **Телесвязь и интернет-инфраструктура продолжают активно развиваться**, что создает спрос на специалистов в этой сфере.  

**4. HR, рекрутинг и консалтинг**  
   - **Кадровые агентства (1 143 вакансии)**,  
   - **Консалтинговые услуги (1 137 вакансий)**.  
   
   Это может говорить о **нехватке кадров на рынке труда** и росте популярности аутсорсинга рекрутинга и консалтинговых услуг.  

**5. Ритейл и e-commerce**  
   - **Розничные сети (1 034 вакансии)**,  
   - **Интернет-магазины (788 вакансий)**.  
   
   В условиях цифровизации **онлайн-торговля продолжает развиваться**, создавая спрос на специалистов в маркетинге, логистике и IT.  

**6. Реклама и маркетинг**  
   - **Маркетинговые и рекламные агентства (1 001 вакансия)**.  
   
   Важность digital-маркетинга и рекламы **не снижается**, что объясняет большое число вакансий в этой сфере.  

**7. Производство и технические специальности**  
   - **Электронно-вычислительная, оптическая, контрольно-измерительная техника, радиоэлектроника, автоматика**  
     - Продвижение, оптовая торговля (622 вакансии).  
     - Производство (584 вакансии).  
   Хотя технические и инженерные вакансии не занимают лидирующие позиции, **спрос на специалистов в этих областях остается стабильным**.  

### **Основные выводы:**  
 **IT — лидер по количеству вакансий.** Разработка ПО, системная интеграция и интернет-сервисы создают основную часть спроса.  
 **Финансовый и телекоммуникационный сектор** также активно нанимает специалистов.  
 **Рынок труда испытывает нехватку специалистов**, что подтверждается спросом на кадровые агентства и консалтинговые услуги.  
 **E-commerce, маркетинг и ритейл развиваются**, создавая новые рабочие места.  
 **Технические и инженерные вакансии есть, но их меньше, чем IT-позиций.**  


Этот анализ показывает ключевые тенденции на рынке труда и может быть полезен как для работодателей, так и для соискателей. 

### 2. В каких отраслях самый высокий средний уровень зарплат?

In [44]:
# SQL-запрос для определения отраслей с самым высоким средним уровнем зарплат
query =  '''
    SELECT  
        i.name AS name,  -- Название отрасли
        ROUND(AVG(
            COALESCE((v.salary_from + v.salary_to) / 2, v.salary_from, v.salary_to)
        )) AS avg_salary  -- Средняя зарплата (берем среднее значение, если указаны оба диапазона)
    FROM employers e
    JOIN employers_industries ei ON e.id = ei.employer_id 
    JOIN industries i ON ei.industry_id = i.id
    JOIN vacancies v ON v.employer_id = e.id
    WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL  -- Фильтр: учитывать только вакансии, где указана зарплата
    GROUP BY 1  -- Группировка по названию отрасли
    ORDER BY 2 DESC  -- Сортировка по убыванию средней зарплаты
    LIMIT 20  -- Выводим топ-20 отраслей
'''

# Выполнение запроса и загрузка данных в DataFrame
df = pd.read_sql_query(query, engine)

# Вывод данных в удобном формате
for index, row in df.iterrows():
    print(f"{index + 1}. {row['name']} ({row['avg_salary']} руб.)")


1. Агрохимия (продвижение, оптовая торговля) (381429.0 руб.)
2. Агрохимия (производство) (339875.0 руб.)
3. Игорный бизнес (191905.0 руб.)
4. Корма для животных (производство) (168300.0 руб.)
5. Зоомагазин (167613.0 руб.)
6. Кадровые агентства (144530.0 руб.)
7. Лакокрасочная продукция, сырье (производство) (142803.0 руб.)
8. Бытовая химия, парфюмерия, косметика  (производство) (130120.0 руб.)
9. Лакокрасочная продукция, сырье (продвижение, оптовая торговля) (128445.0 руб.)
10. Дорожно-строительная техника, сельскохозяйственная и другая спец.техника, оборудование, лифты, погрузочно-разгрузочное, складское оборудование (производство) (122878.0 руб.)
11. Дорожно-строительная техника, сельскохозяйственная и другая спец.техника, оборудование, лифты, погрузочно-разгрузочное, складское оборудование (продвижение, оптовая торговля) (119919.0 руб.)
12. Оборудование для пищевой промышленности, упаковки и хранения (производство) (119656.0 руб.)
13. Финансово-кредитное посредничество (биржа, броке

###  **Анализ данных по уровню зарплат в отраслях**  

---

#### **В каких отраслях самый высокий средний уровень зарплат?**
Наибольши средний уровень зарплат выявлен в следующих отраслях:  
1. **Агрохимия (оптовая торговля)** – 381,429 руб.  
2. **Агрохимия (производство)** – 339,875 руб.  
3. **Игорный бизнес** – 191,905 руб.  
4. **Производство кормов для животных** – 168,300 руб.  
5. **Зоомагазины** – 167,613 руб.  
6. **Кадровые агентства** – 144,530 руб.  
7. **Лакокрасочная продукция (производство)** – 142,803 руб.  
8. **Бытовая химия, парфюмерия, косметика (производство)** – 130,120 руб.  
9. **Финансово-кредитное посредничество** – 117,714 руб.  
10. **Разработка программного обеспечения** – 117,659 руб.  

 **Вывод:** Самые высокие зарплаты наблюдаются в специфических секторах, связанных с агрохимией, производством, игорным бизнесом и финансовым сектором.

---

###  **Совместный анализ по востребованности специалистов и уровню зарплат в отраслях**
1. **IT-сфера (разработка ПО, системная интеграция, интернет-компании)**  
   - Одна из самых востребованных отраслей по количеству вакансий.  
   - Средняя зарплата в **разработке ПО** – **117,659 руб.**  
   - **Вывод**: Вакансий в IT очень много, а уровень зарплат один из высоких, что подтверждает высокий спрос на специалистов.  

2. **Финансово-кредитный сектор (банки, биржи, брокеры и пр.)**  
   - Банки на **4-м месте по числу вакансий**.  
   - Средняя зарплата **117,714 руб.** (высокий уровень).  
   - **Вывод**: Финансовый сектор предлагает хорошие зарплаты и имеет стабильную потребность в кадрах.  

3. **Кадровые агентства**  
   - Вошли в **топ-10 по количеству вакансий** (1,143).  
   - Средняя зарплата **144,530 руб.** (одна из самых высоких).  
   - **Вывод**: Высокая зарплата может быть связана с тем, что кадровые агентства в большинстве случаев размещают вакансии от имени других компаний и часто ищут высококвалифицированных специалистов и топ-менеджеров, где зарплаты выше среднего. Исключение — если агентство ищет сотрудников для себя (например, рекрутеров, HR-менеджеров). Но таких вакансий мало по сравнению с общим объемом размещаемых ими объявлений.  

4. **Мобильная связь**  
   - Входит в топ-5 по количеству вакансий.  
   - Отсутствует в списке с высокими зарплатами.  
   - **Вывод**: Отрасль предлагает много рабочих мест, но, возможно, с невысокими зарплатами по сравнению с IT и финансами.  

---

###  **Общий вывод**
- **IT и финансы** – **лучшие отрасли** по сочетанию востребованности и уровня зарплат.  
- **Агрохимия и игорный бизнес** – имеют **высокие зарплаты**, но относительно небольшое количество вакансий (не вошли в  двадцатку).  
- **Кадровые агентства** – неожиданно высокая зарплата, вероятно, из-за комиссий и премий за подбор специалистов.  
- **Телекоммуникации** (мобильная связь, интернет-провайдеры) – активно ищу сотрудников, но зарплаты, вероятно, ниже средних по другим востребованным отраслям.  

---


## Варианты возможных направлений дополнительных исследований 

### **1. Более глубокий анализ зарплат**  
- Распределение зарплат не только по отраслям, ни и по регионам.  
- Влияние навыков (скилов), а так же опыта на уровень зарплаты в разных профессиях. 
- Разница в зарплатах между вакансиями в зависимости от типа рабочего графика и трудоустройства, а также их сочения.

### **2. Изменение рынка труда во времени**  
- Динамика роста или спада количества вакансий за определенный период.  
- Как изменяются требования работодателей к опыту и навыкам со временем.  
- Сезонность в размещении вакансий. 

### **3. Более глубокий анализ ключевых навыков, вообще, а также вакансий для стажеров и молодых специалистов**  
- Какие навыки чаще всего встречается в требованиях (например, Python, SQL, Excel, HTML...).   
- Какие редкие навыки ценятся больше (возможно, они встречаются только в высокооплачиваемые вакансиях). 
- Насколько гибко компании готовы рассматривать кандидатов без опыта или с частично подходящими навыками.  
- В каких отраслях компании чаще всего нанимают новичков. 
- Какие компании чаще предлагают стажировки.  
- Как быстро растет зарплата у начинающих специалистов. 
- В каких вакансиях обязательное требование — высшее образование. 

Это только часть направлений возможных дополнительных исследований рынка труда, все зависит от специфики целей и задач проводимого исследования.